<a href="https://colab.research.google.com/github/Valeeew/Analyze-bookstore-sales/blob/turnover/Turnover_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse du CA
## Par Valentin MONTEIRO

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!git clone -l -s git://github.com/Valeeew/Analyze-bookstore-sales cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 4), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (12/12), 15.34 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/cloned-repo
customers.csv	  products.csv	       transactions.csv
DATA_library.csv  Sales_analyze.ipynb


In [3]:
df = pd.read_csv("DATA_library.csv", index_col="date", parse_dates=True)

In [4]:
df

,Unnamed: 0,id_prod,session_id,client_id,price,categ,sex,birth
date,,,,,,,,
2022-05-20 13:21:29.043970,0,0_1518,s_211425,c_103,4.18,0.0,f,1986
2022-06-18 05:55:31.816994,1,0_1627,s_225494,c_103,3.99,0.0,f,1986
2023-02-08 17:31:06.898425,2,0_1104,s_338755,c_103,13.21,0.0,f,1986
2021-04-20 06:18:39.939761,3,0_1390,s_23078,c_103,19.37,0.0,f,1986
2022-03-15 19:54:53.904123,4,0_1547,s_179629,c_103,8.99,0.0,f,1986
...,...,...,...,...,...,...,...,...
2021-05-15 08:17:35.835342,679327,0_1509,s_34762,c_8114,4.99,0.0,m,1962
2021-11-08 17:12:27.637349,679328,1_152,s_116675,c_240,11.06,1.0,m,1998
2021-06-01 04:31:16.088314,679329,2_207,s_42418,c_305,50.99,2.0,f,1996


### Objectifs :

> - Indicateurs et graphiques autour du chiffre d'affaires.
- Évolution dans le temps et mise en place d'une décomposition en moyenne mobile pour évaluer la tendance globale
- Zoom sur les références:
 - tops et les flops
 - la répartition par catégorie
- Repérer des informations sur les profils des clients:
 - la répartition du chiffre d'affaires entre eux, via par exemple une courbe de Lorenz.
- Après, toutes les informations et tous graphiques qui apporteraient de l’information pertinente sont les bienvenus !